In [1]:
# DO NOT RUN BECAUSE IT IS FUCKED UP


import os
from concurrent.futures import ThreadPoolExecutor
from typing import List, Tuple

import gc
import re
import sqlite3
import pandas as pd
import tensorflow as tf
from scipy.special import softmax
from sqlalchemy import create_engine
from tqdm import tqdm
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [2]:

def fetch_data(query: str, path: str) -> pd.DataFrame:
    with sqlite3.connect(path) as connection:
        return pd.read_sql_query(query, connection, index_col='tweet_id')


def process_batch(texts):
    """
    Apply sentiment analysis to a batch of texts using a pre-trained transformer model.

    Parameters:
    texts (list): A list of texts to analyze.

    Returns:
    list: A list of ranked sentiment labels for the input texts.

    This function uses a pre-trained transformer model for sequence classification to analyze the sentiment of the texts in the given batch. It returns a list of ranked sentiment labels for the input texts, where the first label is the most likely sentiment and the subsequent labels are less likely sentiments.
    """
    # Get the maximum sequence length for the model

    encoded_input = tokenizer(texts, return_tensors="tf", padding=True, truncation=True)
    with tf.device(device):
        output = model(encoded_input)

    scores = output[0].numpy()
    scores = softmax(scores, axis=1)

    sentiment_scores = scores[:, 2] - scores[:, 0]
    return sentiment_scores.tolist()

# Clear GPU memory function
def clear_gpu_memory():
    tf.keras.backend.clear_session()  # Clear the current session
    try:
        tf.compat.v1.reset_default_graph()  # For TensorFlow 1.x compatibility
    except AttributeError:
        pass
    gc.collect()  # Explicitly call the garbage collector


def apply_sentiment_analysis(df, text_column, batch_size=128, max_workers=4):
    """
    Apply sentiment analysis to the given DataFrame using a pre-trained transformer model.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the text column to analyze.
    text_column (str): The name of the column in the DataFrame containing the text to analyze.
    batch_size (int): The number of texts to process in each batch. Default is 128.
    max_workers (int): The maximum number of worker threads to use for parallel processing. Default is 4.

    Returns:
    pd.DataFrame: The input DataFrame with an additional 'sentiment' column containing the sentiment analysis results.

    This function uses a pre-trained transformer model for sequence classification to analyze the sentiment of the texts in the given DataFrame. It applies the sentiment analysis in parallel using multiple worker threads to improve performance. The results are then added to the input DataFrame as a new 'sentiment' column.
    """
    texts = df[text_column].tolist()
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for i in range(0, len(texts), batch_size):
            batch = texts[i : i + batch_size]
            results.extend(executor.submit(process_batch, batch).result())
            clear_gpu_memory()

    df["sentiment"] = results
    return df


def get_batches(df: pd.DataFrame, batch_size: int = 1000) -> List[pd.DataFrame]:
    """
    Split DataFrame into batches of DataFrames with specified batch size.

    Args:
        df: The DataFrame containing tweet data.
        batch_size: The size of each batch.

    Returns:
        A list of DataFrames, each containing a batch of rows.
    """
    return [df.iloc[i : i + batch_size] for i in range(0, len(df), batch_size)]

def convert_to_list(df: pd.DataFrame) -> List[List]:
    """
    Convert DataFrame with tweet_id as index to a list of lists containing sentiment and tweet_id.

    Args:
        df: The DataFrame with tweet_id as index and sentiment as a column.

    Returns:
        A list of lists containing tweet_id and sentiment.
    """
    tweet_ids = df.index.to_numpy()
    sentiments = df["sentiment"].to_numpy()
    return tuple(zip(sentiments, tweet_ids))

def connect_to_local_database(db_path: str):
    """
    Establish a connection to the local SQLite database.

    Args:
        db_path: The path to the SQLite database file.

    Returns:
        A connection object to the SQLite database.
    """
    try:
        return sqlite3.connect(db_path)
    except sqlite3.Error as e:
        print(f"Error while connecting to SQLite: {e}")
    return None


def update_text_local(
    batch: List[Tuple[str, str]], db_path: str
) -> None:
    """
    Update full_text values for a batch of data in the local SQLite database.

    Args:
        batch: List of (full_text, tweet_id) pairs.
        db_path: The path to the SQLite database file.
    """
    connection = connect_to_local_database(db_path)
    if connection is None:
        return
    try:
        cursor = connection.cursor()
        update_query = "UPDATE Tweets SET sentiment_score = ? WHERE tweet_id = ?"
        cursor.executemany(update_query, batch)
        connection.commit()
    except sqlite3.Error as e:
        print(f"Error updating batch: {e}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [3]:
print("Start the upload")
path = os.path.join(
        os.path.dirname(
            os.getcwd()
        ),
    "data_processed", "local_backup.db")
query = """
SELECT tweet_id, full_text
FROM Tweets
WHERE sentiment_score IS NULL;
"""
batch_size = 5_000
# Load the tokenizer and model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-sentiment"
)
# Check if GPU is available
device = "/GPU:0" if tf.test.is_gpu_available() else "/CPU:0"
if gpus := tf.config.experimental.list_physical_devices('GPU'):
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Start the upload


C:\My programs\Anaconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Physical devices cannot be modified after being initialized


In [4]:
test_data = fetch_data(query, path)

In [5]:
# Airlines list
airlines = [
    'KLM', "British_Airways", "airfrance", "AmericanAir", "lufthansa", 
    "airberlinAssist", "easyJet", "Ryanair", "SingaporeAir", "Qantas",
    "EtihadAirways", "VirginAtlantic", "airberlin"
]

# Function to clean mentions not in the airlines list
def clean_mentions(text):
    # Regex pattern to find mentions
    mention_pattern = r'@([A-Za-z0-9_]+)'
    mentions = re.findall(mention_pattern, text)
    
    # Check if each mention is in the airlines list
    valid_mentions = [f"@{mention}" for mention in mentions if mention in airlines]
    
    # Replace invalid mentions in the text
    for mention in mentions:
        if mention not in airlines:
            text = text.replace(f"@{mention}", "")
    
    return text

# Apply the cleaning function to the DataFrame
test_data['cleaned_text'] = test_data['full_text'].apply(clean_mentions)

In [6]:
test_data

,full_text,cleaned_text
tweet_id,,
1137380899622858752,Prove it. Receipts please. (Not saying I don't...,Prove it. Receipts please. (Not saying I don't...
1137380903548792832,RT @AdaraCensored: .@Delta @AmericanAir @JetBl...,RT : . @AmericanAir \nI am very concerned th...
1137380905792745473,RT @kjbelfon: I think we (especially gay men) ...,RT : I think we (especially gay men) need to d...
1137380911178276864,RT @emcjayyy: Next time you say “sO wHeN iS sT...,RT : Next time you say “sO wHeN iS sTrAiGhT pR...
1137380913837420544,RT @GovMikeHuckabee: In CO Springs and thanks ...,RT : In CO Springs and thanks to @AmericanAir ...
...,...,...
1244696703690772485,RT @jfergo86: Me parece a mí o el avión es más...,RT : Me parece a mí o el avión es más grande q...
1244696708983984131,Today’s random pic of the day is the one of Vo...,Today’s random pic of the day is the one of Vo...
1244696710447800320,RT @SchipholWatch: @spbverhagen @markduursma @...,RT : @KLM Nog niet aan de orde? Als in: e...


In [7]:
data_batches = get_batches(test_data[["cleaned_text"]], batch_size)

In [8]:
#convert_to_list(data_batches[0])
#sentiment_with_score = apply_sentiment_analysis(data_batches[0], "cleaned_text", 16, 14)
#sentiment_with_score

In [9]:
for batch in tqdm(data_batches, desc="Updating text: "):
    df_sentiment = apply_sentiment_analysis(batch, "cleaned_text", 8, 4)
    update_text_local(convert_to_list(df_sentiment), path)

Updating text:   0%|▏                                                            | 3/1158 [18:25<118:16:47, 368.66s/it]


KeyboardInterrupt: 